In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import multiprocessing
import parmap
import numpy as np

In [2]:
biomart_ensembl = pd.read_csv('/home/weber/PycharmProjects/ExoCarto/data/0_external/BIOMART_HGNC_ENSEMBL.txt.gz', sep='\t', compression='gzip')
biomart_ensembl = biomart_ensembl.dropna(subset=['HGNC ID'])
biomart_ensembl['HGNC ID'] = biomart_ensembl['HGNC ID'].astype(str)
biomart_ensembl['HGNC ID'] = biomart_ensembl['HGNC ID'].str.replace('HGNC:', '')
biomart_ensembl

,Gene stable ID,HGNC ID,HGNC symbol
0,ENSG00000210049,7481,MT-TF
1,ENSG00000211459,7470,MT-RNR1
2,ENSG00000210077,7500,MT-TV
3,ENSG00000210082,7471,MT-RNR2
4,ENSG00000209082,7490,MT-TL1
...,...,...,...
67150,ENSG00000288200,49825,SNRPCP10
67152,ENSG00000288476,35781,RPS3AP15
67155,ENSG00000284997,14531,TUBAP2
67156,ENSG00000284805,47786,RNU6-823P


In [4]:
ptex_file = "/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad.parquet"
rawdf_ptex = pd.read_parquet(ptex_file)
rawdf_ptex = pd.merge(rawdf_ptex, biomart_ensembl, right_on='HGNC symbol', left_on='symbol')
rawdf_ptex = rawdf_ptex[['HGNC ID', 'symbol', 'ensg', 'chrom', 'pos'] + list(rawdf_ptex.columns)[4:-3]]
rawdf_ptex['HGNC ID'] = rawdf_ptex['HGNC ID'].astype(int)
rawdf_ptex

,HGNC ID,symbol,ensg,chrom,pos,Cells_Transformedfibroblasts,Prostate,Spleen,Brain_FrontalCortex_BA9_,SmallIntestine_TerminalIleum,...,Brain_Anteriorcingulatecortex_BA24_,Brain_Putamen_basalganglia_,Brain_Caudate_basalganglia_,Colon_Sigmoid,Cervix_Endocervix,Ovary,Esophagus_GastroesophagealJunction,Testis,Brain_Amygdala,mean_proportion
0,28706,SAMD11,ENSG00000187634,1,861322,0.24176,0.21392,0.22348,0.21429,0.23819,...,0.15217,0.14238,0.12476,0.26381,0.14811,0.12727,0.27732,0.25311,0.00000,0.16993
1,28706,SAMD11,ENSG00000187634,1,861323,0.24176,0.21392,0.22348,0.21429,0.23819,...,0.15217,0.14238,0.12476,0.26381,0.14811,0.12727,0.27732,0.25311,0.00000,0.16993
2,28706,SAMD11,ENSG00000187634,1,861324,0.24176,0.21392,0.22348,0.21429,0.23819,...,0.15217,0.14238,0.12476,0.26381,0.14811,0.12727,0.27732,0.25311,0.00000,0.16993
3,28706,SAMD11,ENSG00000187634,1,861325,0.24176,0.21392,0.22348,0.21429,0.23819,...,0.15217,0.14238,0.12476,0.26381,0.14811,0.12727,0.27732,0.25311,0.00000,0.16993
4,28706,SAMD11,ENSG00000187634,1,861326,0.24176,0.21392,0.22348,0.21429,0.23819,...,0.15217,0.14238,0.12476,0.26381,0.14811,0.12727,0.27732,0.25311,0.00000,0.16993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27005443,2699,DDX3Y,ENSG00000067048,Y,15030030,0.75703,0.66515,0.80067,0.86645,0.66719,...,0.90024,0.88398,0.86485,0.76432,1.00000,1.00000,0.73750,0.71384,0.89037,0.77838
27005444,2699,DDX3Y,ENSG00000067048,Y,15030031,0.75703,0.66515,0.80067,0.86645,0.66719,...,0.90024,0.88398,0.86485,0.76432,1.00000,1.00000,0.73750,0.71384,0.89037,0.77838
27005445,2699,DDX3Y,ENSG00000067048,Y,15030032,0.75703,0.66515,0.80067,0.86645,0.66719,...,0.90024,0.88398,0.86485,0.76432,1.00000,1.00000,0.73750,0.71384,0.89037,0.77838
27005446,2699,DDX3Y,ENSG00000067048,Y,15030033,0.75703,0.66515,0.80067,0.86645,0.66719,...,0.90024,0.88398,0.86485,0.76432,1.00000,1.00000,0.73750,0.71384,0.89037,0.77838


In [8]:
len(sorted(list(rawdf_ptex.columns))[:-5])

54

In [9]:
original_pext = pd.read_csv('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/all.baselevel.021620.tsv.bgz', compression='gzip', sep='\t', low_memory=False)
pext_modified = original_pext.dropna(subset=[list(original_pext.columns)[-1]])
pext_modified[['chrom', 'pos']] = pext_modified['locus'].str.split(':', expand=True)
pext_modified = pd.merge(pext_modified, biomart_ensembl, right_on='HGNC symbol', left_on='symbol', how='left')



/home/weber/.local/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [31]:
pext_cols = sorted(list(pext_modified.columns)[3:-5])
others = sorted([e for e in list(pext_modified.columns) if e not in pext_cols])
pext_modified2 = pext_modified[others + pext_cols]
# pext_modified = pext_modified[['HGNC ID', 'symbol', 'ensg', 'chrom', 'pos'] + list(pext_modified.columns)[4:-3]]
# original_pext['HGNC ID'] = original_pext['HGNC ID'].astype(int)
pext_modified2

,Gene stable ID,HGNC ID,HGNC symbol,chrom,ensg,locus,pos,symbol,Adipose_Subcutaneous,Adipose_Visceral_Omentum_,...,Skin_SunExposed_Lowerleg_,SmallIntestine_TerminalIleum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,WholeBlood,mean_proportion
0,NaN,NaN,NaN,1,ENSG00000237683,1:138530,138530,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
1,NaN,NaN,NaN,1,ENSG00000237683,1:138531,138531,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
2,NaN,NaN,NaN,1,ENSG00000237683,1:138532,138532,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
3,NaN,NaN,NaN,1,ENSG00000237683,1:138533,138533,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
4,NaN,NaN,NaN,1,ENSG00000237683,1:138534,138534,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38837009,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770670,27770670,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158
38837010,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770671,27770671,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158
38837011,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770672,27770672,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158
38837012,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770673,27770673,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158


In [37]:
list(pext_modified2.columns)[8:]

['Adipose_Subcutaneous',
 'Adipose_Visceral_Omentum_',
 'AdrenalGland',
 'Artery_Aorta',
 'Artery_Coronary',
 'Artery_Tibial',
 'Bladder',
 'Brain_Amygdala',
 'Brain_Anteriorcingulatecortex_BA24_',
 'Brain_Caudate_basalganglia_',
 'Brain_CerebellarHemisphere',
 'Brain_Cerebellum',
 'Brain_Cortex',
 'Brain_FrontalCortex_BA9_',
 'Brain_Hippocampus',
 'Brain_Hypothalamus',
 'Brain_Nucleusaccumbens_basalganglia_',
 'Brain_Putamen_basalganglia_',
 'Brain_Spinalcord_cervicalc_1_',
 'Brain_Substantianigra',
 'Breast_MammaryTissue',
 'Cells_EBV_transformedlymphocytes',
 'Cells_Transformedfibroblasts',
 'Cervix_Ectocervix',
 'Cervix_Endocervix',
 'Colon_Sigmoid',
 'Colon_Transverse',
 'Esophagus_GastroesophagealJunction',
 'Esophagus_Mucosa',
 'Esophagus_Muscularis',
 'FallopianTube',
 'Heart_AtrialAppendage',
 'Heart_LeftVentricle',
 'Kidney_Cortex',
 'Liver',
 'Lung',
 'MinorSalivaryGland',
 'Muscle_Skeletal',
 'Nerve_Tibial',
 'Ovary',
 'Pancreas',
 'Pituitary',
 'Prostate',
 'Skin_NotSunExp

In [39]:
pext_modified2

,Gene stable ID,HGNC ID,HGNC symbol,chrom,ensg,locus,pos,symbol,Adipose_Subcutaneous,Adipose_Visceral_Omentum_,...,Skin_SunExposed_Lowerleg_,SmallIntestine_TerminalIleum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,WholeBlood,mean_proportion
0,NaN,NaN,NaN,1,ENSG00000237683,1:138530,138530,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
1,NaN,NaN,NaN,1,ENSG00000237683,1:138531,138531,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
2,NaN,NaN,NaN,1,ENSG00000237683,1:138532,138532,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
3,NaN,NaN,NaN,1,ENSG00000237683,1:138533,138533,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
4,NaN,NaN,NaN,1,ENSG00000237683,1:138534,138534,AL627309.1,1.0,1.0,...,1.0,1.0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38837009,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770670,27770670,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158
38837010,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770671,27770671,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158
38837011,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770672,27770672,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158
38837012,ENSG00000172288,1809,CDY1,Y,ENSG00000172288,Y:27770673,27770673,CDY1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.26316,NaN,NaN,NaN,NaN,0.13158


In [38]:
pext_modified.loc[:,~pext_modified.columns.duplicated()]

,ensg,locus,symbol,Cells_Transformedfibroblasts,Prostate,Spleen,Brain_FrontalCortex_BA9_,SmallIntestine_TerminalIleum,MinorSalivaryGland,Artery_Coronary,...,Ovary,Esophagus_GastroesophagealJunction,Testis,Brain_Amygdala,mean_proportion,chrom,pos,Gene stable ID,HGNC ID,HGNC symbol
0,ENSG00000237683,1:138530,AL627309.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.00000,1,138530,NaN,NaN,NaN
1,ENSG00000237683,1:138531,AL627309.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.00000,1,138531,NaN,NaN,NaN
2,ENSG00000237683,1:138532,AL627309.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.00000,1,138532,NaN,NaN,NaN
3,ENSG00000237683,1:138533,AL627309.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.00000,1,138533,NaN,NaN,NaN
4,ENSG00000237683,1:138534,AL627309.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.00000,1,138534,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38837009,ENSG00000172288,Y:27770670,CDY1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.26316,NaN,0.13158,Y,27770670,ENSG00000172288,1809,CDY1
38837010,ENSG00000172288,Y:27770671,CDY1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.26316,NaN,0.13158,Y,27770671,ENSG00000172288,1809,CDY1
38837011,ENSG00000172288,Y:27770672,CDY1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.26316,NaN,0.13158,Y,27770672,ENSG00000172288,1809,CDY1
38837012,ENSG00000172288,Y:27770673,CDY1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.26316,NaN,0.13158,Y,27770673,ENSG00000172288,1809,CDY1


In [40]:
pext_modified2.to_parquet('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad_corrected.parquet')

In [ ]:
pext_modified.loc[:,~pext_modified.columns.duplicated()].to_parquet('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad_corrected.parquet')

In [ ]:
pext_modified = pd.read_parquet('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad_corrected.parquet')
pext_modified

In [41]:
pext_modified = pext_modified2

In [ ]:
pext_modified.columns

In [42]:
pext_modified = pext_modified.dropna(subset=['HGNC ID'])
pext_modified['HGNC ID'] = pext_modified['HGNC ID'].astype(int)
pext_modified['pos'] = pext_modified['pos'].astype(int)


/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
refseq = pd.read_csv('/home/weber/PycharmProjects/ExoCarto/data/2_processed/DF_transformed.csv.gz', compression='gzip', sep='\t').dropna(subset=['HGNC']).sort_values(by=['HGNC', 'Chrom', 'Start'], ascending=True)
refseq['HGNC'] = refseq['HGNC'].astype(int)
refseq

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Gene,Chrom,Start,End,ranges,Length,GeneID,HGNC,CDS_representation,Ratio,Sharing_status,Sources,Count_CDS_alternative,Count_CDS_constitutive,mRNA_IDS,mRNA_nb
110056,A1BG,19,58858388,58858395,58858388-58858395,7,1,5,Unique,1.0/1,False,"[(58858388, 58858395, '1.0/1')]",0,8,['NM_130786.4'],1
110057,A1BG,19,58858719,58859006,58858719-58859006,287,1,5,Unique,1.0/1,False,"[(58858719, 58859006, '1.0/1')]",0,8,['NM_130786.4'],1
110058,A1BG,19,58861736,58862017,58861736-58862017,281,1,5,Unique,1.0/1,False,"[(58861736, 58862017, '1.0/1')]",0,8,['NM_130786.4'],1
110059,A1BG,19,58862757,58863053,58862757-58863053,296,1,5,Unique,1.0/1,False,"[(58862757, 58863053, '1.0/1')]",0,8,['NM_130786.4'],1
110060,A1BG,19,58863649,58863921,58863649-58863921,272,1,5,Unique,1.0/1,False,"[(58863649, 58863921, '1.0/1')]",0,8,['NM_130786.4'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79640,DERPC,16,69152757,69154010,69152757-69154010,1253,113455421,54084,Variable_region,4.0/7,True,"[(69152757, 69154331, '4.0/7')]",2,1,"['NM_001366605.1', 'NM_001366603.1', 'NM_00104...",7
79641,DERPC,16,69154010,69154331,69154010-69154331,321,113455421,54084,Variable_region,7.0/7,True,"[(69152757, 69154331, '4.0/7'), (69154010, 691...",2,1,"['NM_001366605.1', 'NM_001366603.1', 'NM_00104...",7
79642,DERPC,16,69154010,69154331,69154010-69154331,321,113455421,54084,Variable_region,7.0/7,True,"[(69152757, 69154331, '4.0/7'), (69154010, 691...",2,1,"['NM_001366605.1', 'NM_001366603.1', 'NM_00104...",7
9453,LINC02801,1,87709298,87709320,87709298-87709320,22,101927844,54325,Unique,1.0/1,False,"[(87709298, 87709320, '1.0/1')]",0,2,['NM_001290053.1'],1


In [44]:
multi_mrna_genes = refseq.loc[refseq['mRNA_nb'] > 1, 'HGNC'].unique()
multi_mrna_genes

array([    7,    18,    19, ..., 53651, 53757, 54084])

In [45]:



def map_refseq_pext(gene, l):
    tmp_l = list()
    refseq_gene = refseq.loc[refseq['HGNC'] == gene].reset_index(drop=True)
    pext_gene = pext_modified.loc[pext_modified['HGNC ID'] == gene].reset_index(drop=True)
    for r in refseq_gene[['Start', 'End']].values:
        start = r[0]
        end = r[1]
        pext_exon_values = pext_gene.loc[pext_gene['pos'].between(start, end)].groupby(['HGNC ID', 'symbol', 'ensg']).median().drop(['pos'], axis=1)
        pext_exon_values['Exon'] = str(start) + '-' + str(end)
        tmp_l.append(pext_exon_values)
    if tmp_l:
        concat_df_gene = pd.concat(tmp_l).reset_index()
        concat_df_gene = concat_df_gene[list(concat_df_gene.columns[:3]) + ['Exon'] + sorted(list(concat_df_gene.columns[3:-2])) + ['mean_proportion']]
        l.append(concat_df_gene)
    
    
m = multiprocessing.Manager()
l = m.list()
parmap.starmap(map_refseq_pext, list(zip(sorted(refseq.HGNC.unique()))), l, pm_pbar=True, pm_processes=20)
    
output_df = pd.concat(list(l))
output_df.to_parquet('/home/weber/PycharmProjects/ExoCarto/data/clean/2_expression/RefSeq_PEXT_2021.parquet', index=False)
output_df



    # transposed = test.T
    # percentile_df = list()
    # for col in transposed:
    #     percentile_df.append(transposed[col].rank(pct=True))
    # pd.concat(percentile_df, axis=1).sort_index().T




19200it [05:17, 60.48it/s]                            


,HGNC ID,symbol,ensg,Exon,Adipose_Subcutaneous,Adipose_Visceral_Omentum_,AdrenalGland,Artery_Aorta,Artery_Coronary,Artery_Tibial,...,Skin_SunExposed_Lowerleg_,SmallIntestine_TerminalIleum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,WholeBlood,mean_proportion
0,4291.0,GK2,ENSG00000196475,80327693-80329354,NaN,NaN,NaN,NaN,NaN,1.00000,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000
0,1109.0,BRI3,ENSG00000164713,97911108-97911249,0.58244,0.58290,0.62426,0.61856,0.59839,0.59585,...,0.55253,0.60333,0.61408,0.56897,0.489850,0.59181,0.61988,0.61228,0.54971,0.581680
1,1109.0,BRI3,ENSG00000164713,97911663-97911765,0.58244,0.58290,0.62426,0.61856,0.59839,0.59585,...,0.55253,0.60333,0.61408,0.56897,0.489850,0.59181,0.61988,0.61228,0.54971,0.581680
2,1109.0,BRI3,ENSG00000164713,97920423-97920555,0.58244,0.58290,0.62426,0.61856,0.59839,0.59585,...,0.55253,0.60333,0.61408,0.56897,0.489850,0.59181,0.61988,0.61228,0.54971,0.581680
3,1109.0,BRI3,ENSG00000164713,97921951-97922002,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,29197.0,CCSER2,ENSG00000107771,86259631-86259715,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.033532,0.51133,0.62554,0.56584,0.50833,0.668990
11,29197.0,CCSER2,ENSG00000107771,86259715-86259763,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.033532,0.51133,0.62554,0.56584,0.50833,0.668990
12,29197.0,CCSER2,ENSG00000107771,86273205-86273268,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.034523,0.51133,0.62554,0.56584,0.50833,0.672050
13,29197.0,CCSER2,ENSG00000107771,86273268-86273299,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.034523,0.51133,0.62554,0.56584,0.50833,0.672050
